In [25]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [26]:
serv=Service("C:/Users/mohit.chaniyal/Documents/chromedriver.exe") #provide driver location here

In [27]:
# with open("myinstacred.txt","r") as f:
#     uname,passwd=[x.replace("\n","") for x in f.readlines()]
uname="username"  
passwd="password"

In [28]:
mobile_emulation = {"deviceName": "iPhone X"}
chrome_options = ChromeOptions()
chrome_options.add_experimental_option("mobileEmulation", mobile_emulation)
driver =Chrome(options=chrome_options,service=serv)
driver.get("https://www.instagram.com/")
driver.set_window_size(375, 812)  # iPhone X resolution: 375x812
#login
time.sleep(2)
driver.find_element(By.XPATH,"//div[text()='Log in']").click()
time.sleep(2)
driver.find_element(By.NAME,"username").send_keys(uname)
driver.find_element(By.NAME,"password").send_keys(passwd)
driver.find_element(By.XPATH,"//div[text()='Log in']").click()

In [34]:
def get_follower_following(username:str,type_:str)->set:
    driver.get(f"https://www.instagram.com/{username}/{type_}/")
    time.sleep(3)
    prev_height=driver.execute_script("return document.body.scrollHeight")
    f_set=set()
    while True :
        f_list=driver.find_elements(By.XPATH,"//span[@class='_aacl _aaco _aacw _aacx _aad7 _aade']/div")
        for f in f_list :
            if f.text not in f_set:
                f_set.add(f.text)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(3)
        new_height=driver.execute_script("return document.body.scrollHeight")
        if new_height==prev_height:
            break
        prev_height=new_height
    return f_set

In [35]:
#scraping all the instagram followers
followings=get_follower_following(username="user_name",type_="following")
followers=get_follower_following(username="user_name",type_="followers")


In [43]:
#saving data to csv files
import pandas as pd
pd.DataFrame(followings,columns=["Following"])["Following"].apply(lambda x : x.replace("\nVerified","")).to_csv("following.csv",index=False)
pd.DataFrame(followers,columns=["Followers"])["Followers"].apply(lambda x : x.replace("\nVerified","")).to_csv("followers.csv",index=False)

In [44]:
df1=pd.read_csv("following.csv") 

In [45]:
df2=pd.read_csv("followers.csv")

In [46]:
person_i_follow_but_they_do_not_follow_me=df1[~df1.Following.isin(df2.Followers)]

In [239]:
pd.set_option('display.max_rows', 200)

In [53]:
#creating another instance becaouse facing issue with the display size while unfollowing
browser =Chrome(service=serv)
browser.get("https://www.instagram.com/")
#login

time.sleep(2)
browser.find_element(By.XPATH,"//div[text()='Log in']").click()
browser.find_element(By.NAME,"username").send_keys(uname)
browser.find_element(By.NAME,"password").send_keys(passwd)
browser.find_element(By.XPATH,"//div[text()='Log in']").click()

In [ ]:
#unfollowing people who dont follow me and are not verified
for person in person_i_follow_but_they_do_not_follow_me.Following[4:]:
    browser.get(f"https://www.instagram.com/{person}")
    time.sleep(2)
    try:
        browser.find_element(By.XPATH,"//span[text()='Verified']")
        continue
    except:
        try :
            browser.find_element(By.XPATH,"//div[text()='Following']").click()
            time.sleep(2)
            browser.find_element(By.XPATH,"//span[text()='Unfollow']").click()
            time.sleep(2)
        except:
            print(pe)